# Load Libaray

In [5]:
!pip install xgboost

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.metrics import r2_score
import keras.optimizers

from keras.layers import Dense,Dropout,SimpleRNN,LSTM
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

plt.rcParams['font.family'] = 'Malgun Gothic'

# 데이터 준비

## Train

In [2]:
# 데이터 load
raw_train= pd.read_csv('./data/한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')
raw_train.head(25)

# 데이터 자료형 전처리
# 구분 확인
raw_train['구분'].unique()

# string object -> datetime 
raw_train['연월일'] = pd.to_datetime(raw_train['연월일'], format = '%Y-%m-%d')

# datetime + 시간 
## 시간 int -> flaot (datetime 더하기 할려면 필요)
raw_train = raw_train.astype({'시간':'float64'})
raw_train['연월일시'] = [raw_train['연월일'][i]+ timedelta(hours = raw_train['시간'][i]-1) for i in range(len(raw_train))]
#print(raw_df.head())

'''
# 연월일시 나눠서 컬럼 생성
raw_df['year'] = [raw_df['연월일시'][i].year for i in range(len(raw_df))]
raw_df['month'] = [raw_df['연월일시'][i].month for i in range(len(raw_df))]
raw_df['day'] = [raw_df['연월일시'][i].day for i in range(len(raw_df))]
raw_df['hour'] = [raw_df['연월일시'][i].hour for i in range(len(raw_df))]
#print(raw_df.tail())
'''
#raw_train

df = raw_train.set_index(raw_train['연월일시']).loc[:, ['구분', '공급량']]
df

,구분,공급량
연월일시,,
2013-01-01 00:00:00,A,2497.129
2013-01-01 01:00:00,A,2363.265
2013-01-01 02:00:00,A,2258.505
2013-01-01 03:00:00,A,2243.969
2013-01-01 04:00:00,A,2344.105
...,...,...
2018-12-31 19:00:00,H,681.033
2018-12-31 20:00:00,H,669.961
2018-12-31 21:00:00,H,657.941


## Test

In [4]:
# test 데이터 load
raw_test = pd.read_csv('./data/test.csv')
raw_test.head()

# 일자, 시간, 구분 분리
raw_test['일자'] = raw_test['일자|시간|구분'].str.split(' ').str[0]
raw_test['시간'] = raw_test['일자|시간|구분'].str.split(' ').str[1].astype(int)
raw_test['구분'] = raw_test['일자|시간|구분'].str.split(' ').str[2]
raw_test

# 원래 column 제거
raw_test.drop(columns='일자|시간|구분', inplace=True)
raw_test

# string object -> datetime 
raw_test['일자'] = pd.to_datetime(raw_test['일자'], format = '%Y-%m-%d')

# datetime + 시간 
## 시간 int -> flaot (datetime 더하기 할려면 필요)
raw_test = raw_test.astype({'시간':'float64'})
raw_test['일자'] = [raw_test['일자'][i]+ timedelta(hours = raw_test['시간'][i]-1) for i in range(len(raw_test))]

raw_test.head(25)

# 시점 데이터 index로 변경
test= raw_test.set_index(raw_test['일자'])

# 컬럼 선택 및 y 생성
test['공급량'] = pd.Series()
test = test.loc[:, ['구분', '공급량']]
test

,구분,공급량
일자,,
2019-01-01 00:00:00,A,NaN
2019-01-01 01:00:00,A,NaN
2019-01-01 02:00:00,A,NaN
2019-01-01 03:00:00,A,NaN
2019-01-01 04:00:00,A,NaN
...,...,...
2019-03-31 19:00:00,H,NaN
2019-03-31 20:00:00,H,NaN
2019-03-31 21:00:00,H,NaN
